In [14]:
csv_name = 'train_ver2'

In [1]:
# Import necessary stuff
import pandas as pd
import os.path
import numpy as np
from joblib import Parallel, delayed
processors = 4
chunksize=10000

In [2]:
# This is my personal data folder. Change to your own
#dir = "/Users/piromast/Documents/Kaggle/SantanderData/"
import os
dir = os.getcwd()
#train_csv = os.path.join(dir,'train_ver2.csv')
client_ids = np.load('client_ids.npy')
client_number = client_ids.shape[0]

In [3]:
import shlex
from subprocess import Popen, PIPE

def get_dataset_lines():
    cmd = "wc -l " + csv_name + '.csv'
    process = Popen(shlex.split(cmd), stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait
    train_lines = int(output.split()[0])
    return train_lines

train_lines = get_dataset_lines()
train_chunks = int(np.ceil(float(train_lines)/chunksize))
print 'train_lines = ',train_lines
print 'train_chunks = ',train_chunks

train_lines =  13647310
train_chunks =  1365


In [5]:
#train_reader = pd.read_csv(train_csv, chunksize=1000)
#train_reader = pd.read_csv(train_csv, iterator=True, chunksize=1000)
#
def get_client_ids():
    client_ids = np.array([],dtype='int')
    for i, chunk in enumerate(train_reader):
        client_ids = np.union1d(client_ids,chunk.ncodpers.unique())
    return client_ids

def create_client_ids_array():
    client_ids = get_client_ids()
    #client_ids.shape
    np.save('client_ids',client_ids)
    
#create_client_ids_array()


In [6]:
class train_chunker(object):
    """
    """
    def __init__(self,chunksize):
        self.chunksize     = chunksize
        self.csv_path      = os.path.join(dir,'train_ver2.csv')
        self.client_ids    = np.load('client_ids.npy')
        self.client_number = self.client_ids.shape[0]

        self.chunks = int(np.ceil(float(self.client_number)/self.chunksize))
        print(str(self.chunks) + ' chunks')
        self.reader = pd.read_csv(self.csv_path, chunksize=self.chunksize)


In [10]:
train_reader = pd.read_csv(csv_name, chunksize=chunksize)


def find_client_index(ncodpers):
    return np.argwhere(client_ids == ncodpers)

def create_dataset_client_index_array():

    train_data_client_indices = np.array([],dtype='int')
    #train_data_client_indices = np.full((client_number),np.NaN,dtype='int')
    j = 0

    # measure process time
    import time
    t0 = time.clock()

    for i, chunk in enumerate(train_reader):
        if i%100==0:
            proc_time = time.clock() - t0
            print('chunk ' + str(i) + ' from ' str(train_chunks) + '. (' + str(proc_time) + ' secs)')

        chunk_ncodpers = chunk.ncodpers
        #print chunk_ncodpers[0]
        stdout = Parallel(n_jobs=processors)\
        (delayed(find_client_index)(ncodpers) for ncodpers in chunk_ncodpers)

        chunk_indices = np.full((chunksize),np.NaN)
        for i, indice in enumerate(stdout):
            chunk_indices[i]= indice

        train_data_client_indices = np.concatenate((train_data_client_indices, chunk_indices))
        #    train_data_client_indices[i*chunksize:(i+1)*chunksize] = chunk_indices

    np.save(csv_name + '__client_indices',train_data_client_indices)

chunk 0
0.059858 seconds process time
chunk 10
12.360833 seconds process time
chunk 20
25.15628 seconds process time
chunk 30
38.177 seconds process time
chunk 40
51.380304 seconds process time
chunk 50
64.405444 seconds process time
chunk 60
78.027614 seconds process time
chunk 70
91.774762 seconds process time
chunk 80
105.387984 seconds process time
chunk 90
119.234759 seconds process time
chunk 100
132.591157 seconds process time
chunk 110
145.685445 seconds process time
chunk 120
159.146393 seconds process time
chunk 130
172.193065 seconds process time
chunk 140
185.709338 seconds process time
chunk 150
199.163887 seconds process time
chunk 160
212.279586 seconds process time
chunk 170
225.538692 seconds process time
chunk 180
238.93178 seconds process time
chunk 190
252.598802 seconds process time
chunk 200
265.682728 seconds process time
chunk 210
279.106672 seconds process time
chunk 220
292.501515 seconds process time
chunk 230
306.255064 seconds process time
chunk 240
320.409

In [ ]:
client_ids[540143]

In [ ]:
np.array(stdout[:])[0]
print stdout[0][0][0]
print stdout[:][0][0]
print stdout[0][:][0]
print stdout[0][0][:]
chunk_indices = np.full((chunksize),np.NaN)
for i, indice in enumerate(stdout):
    chunk_indices[i]= indice
print chunk_indices.shape

In [ ]:
# Import necessary stuff
import pandas as pd
import os.path
import numpy as np

In [ ]:
train_reader = pd.read_csv(train_csv, chunksize=chunksize)
client_id = client_ids[0]
client_matrix = pd.DataFrame()
for i, chunk in enumerate(train_reader):
    client_lines = chunk.ncodpers == client_id
    #print client_lines == True
    client_matrix = client_matrix.append(chunk.ix[client_lines])    
    isinchunk = np.sum(client_lines)

#    if isinchunk != 0.0:
#        print i, isinchunk,len(client_matrix)
#        #print chunk.ix[client_lines]
#        print client_matrix
#        raw_input()
        #break
client_matrix

In [ ]:
for i, chunk in enumerate(train_reader):
    pass
print i
#    print(chunk.ncodpers.unique().shape)

#data=train_df.get_chunk()
#data.ncodpers.unique()
#for chunk_df in train_df:
#    chunk_data = chunk_df.get_chunk()
#    print chunk_data.shape
#read
#data.ncodpers.unique()

In [ ]:
# Import description table (describing the meaning of each column)
description = read_csv(os.path.join(dir,'column_description.csv'),encoding = 'latin1')
description

In [ ]:
# Select product column names and build indeces range
column_names = data.columns.values
product_names = [(i, s) for i, s in enumerate(column_names) if 'ind_' in s and '_ult1' in s]
product_indeces = [e[0] for e in product_names]
mInd = min(product_indeces)
MInd = max(product_indeces)
product_indeces_range = range(mInd,MInd)

In [ ]:
product_names

In [ ]:
# Build a dataframe only with products
product_df = data.iloc[:,product_indeces_range]

In [ ]:
# Build a Series with number of products and use description as indeces
desclist=description.iloc[product_indeces_range,1].tolist()
num_products = Series(product_df.sum().tolist(),index=desclist)
num_products
